# COQ7
Pathogenic variants in COQ7 are associated with [Neuronopathy, distal hereditary motor, autosomal recessive 9](https://omim.org/entry/620402).

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.56


In [2]:
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
created_by="ORCID:0000-0002-0736-9199"
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [3]:
df = pd.read_excel("input/COQ7_individuals.xlsx")
df.head(2)

,PMID,title,individual_id,comment,disease_id,disease_label,transcript,allele_1,allele_2,variant.comment,...,Hoffmann sign,Babinski sign,Disturbed sensory perception,Dementia,Lower limb amyotrophy,Fasciculations,Gowers sign,Increased variability in muscle fiber diameter,Oligohydramnios,Premature birth
0,str,str,str,optional str,str,str,str,str,str,optional str,...,HP:0031993,HP:0003487,HP:0010524,HP:0000726,HP:0007210,HP:0002380,HP:0003391,HP:0003557,HP:0001562,HP:0001622
1,PMID:36454683,Homozygous COQ7 mutation: a new cause of potentially treatable distal hereditary motor neuropathy,II-2,NaN,OMIM:620402,"Neuronopathy, distal hereditary motor, autosomal recessive 9",NM_016138.5,c.3G>T,c.3G>T,"NP_057222.2, p.Met1?",...,observed,excluded,excluded,excluded,observed,excluded,excluded,na,NaN,NaN


In [4]:
encoder = CaseTemplateEncoder(df=df, hpo_cr=hpo_cr, created_by=created_by)
individuals = encoder.get_individuals()

Created encoders for 31 fields


In [5]:
COQ7_transcript = "NM_016138.5"
vmanager = VariantManager(df=df,
                          individual_column_name="individual_id",
                          gene_symbol="COQ7",
                          transcript=COQ7_transcript,
                          allele_1_column_name="allele_1",
                          allele_2_column_name="allele_2")

[INFO] encoding variant "c.1A>G"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_016138.5%3Ac.1A>G/NM_016138.5?content-type=application%2Fjson
[INFO] encoding variant "c.3G>T"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_016138.5%3Ac.3G>T/NM_016138.5?content-type=application%2Fjson


In [6]:
vmanager.to_summary()

,status,count,alleles
0,mapped,2,"c.1A>G, c.3G>T"
1,unmapped,0,


In [7]:
vmanager.add_variants_to_individuals(individuals)

In [8]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1,
                                allelic_requirement=AllelicRequirement.BI_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

In [9]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
II-2 (MALE; P36Y),"Neuronopathy, distal hereditary motor, autosomal recessive 9 (OMIM:620402)",NM_016138.5:c.3G>T (homozygous),P12Y: Difficulty walking (HP:0002355)Distal upper limb muscle weakness (HP:0008959); Distal lower limb muscle weakness (HP:0009053); Pes cavus (HP:0001761); Hyperreflexia (HP:0001347); Hoffmann sign (HP:0031993); Lower limb amyotrophy (HP:0007210); excluded: Proximal muscle weakness in upper limbs (HP:0008997); excluded: Proximal muscle weakness in lower limbs (HP:0008994); excluded: Babinski sign (HP:0003487); excluded: Disturbed sensory perception (HP:0010524); excluded: Dementia (HP:0000726); excluded: Fasciculations (HP:0002380); excluded: Gowers sign (HP:0003391)
II-3 (FEMALE; P25Y),"Neuronopathy, distal hereditary motor, autosomal recessive 9 (OMIM:620402)",NM_016138.5:c.3G>T (homozygous),P9Y: Difficulty walking (HP:0002355)Distal upper limb muscle weakness (HP:0008959); Distal lower limb muscle weakness (HP:0009053); Pes cavus (HP:0001761); Babinski sign (HP:0003487); Gowers sign (HP:0003391); excluded: Proximal muscle weakness in upper limbs (HP:0008997); excluded: Proximal muscle weakness in lower limbs (HP:0008994); excluded: Hyperreflexia (HP:0001347); excluded: Hoffmann sign (HP:0031993); excluded: Disturbed sensory perception (HP:0010524); excluded: Dementia (HP:0000726); excluded: Lower limb amyotrophy (HP:0007210)
II-1 (FEMALE; P42Y),"Neuronopathy, distal hereditary motor, autosomal recessive 9 (OMIM:620402)",NM_016138.5:c.3G>T (homozygous),P10Y: Difficulty walking (HP:0002355)Distal upper limb muscle weakness (HP:0008959); Distal lower limb muscle weakness (HP:0009053); Proximal muscle weakness in lower limbs (HP:0008994); Pes cavus (HP:0001761); Lower limb amyotrophy (HP:0007210); Fasciculations (HP:0002380); Gowers sign (HP:0003391); excluded: Proximal muscle weakness in upper limbs (HP:0008997); excluded: Hyperreflexia (HP:0001347); excluded: Hoffmann sign (HP:0031993); excluded: Babinski sign (HP:0003487); excluded: Disturbed sensory perception (HP:0010524); excluded: Dementia (HP:0000726)
Sibling 1 (MALE; ),"Neuronopathy, distal hereditary motor, autosomal recessive 9 (OMIM:620402)",NM_016138.5:c.1A>G (homozygous),Distal upper limb muscle weakness (HP:0008959); Distal lower limb muscle weakness (HP:0009053); Lower limb amyotrophy (HP:0007210); excluded: Proximal muscle weakness in upper limbs (HP:0008997); excluded: Proximal muscle weakness in lower limbs (HP:0008994); excluded: Disturbed sensory perception (HP:0010524)
Sibling 2 (FEMALE; ),"Neuronopathy, distal hereditary motor, autosomal recessive 9 (OMIM:620402)",NM_016138.5:c.1A>G (homozygous),Distal upper limb muscle weakness (HP:0008959); Distal lower limb muscle weakness (HP:0009053); Lower limb amyotrophy (HP:0007210); Increased variability in muscle fiber diameter (HP:0003557); excluded: Proximal muscle weakness in upper limbs (HP:0008997); excluded: Proximal muscle weakness in lower limbs (HP:0008994); excluded: Disturbed sensory perception (HP:0010524)
Sibling 3 (MALE; ),"Neuronopathy, distal hereditary motor, autosomal recessive 9 (OMIM:620402)",NM_016138.5:c.1A>G (homozygous),Distal upper limb muscle weakness (HP:0008959); Distal lower limb muscle weakness (HP:0009053); Lower limb amyotrophy (HP:0007210); excluded: Proximal muscle weakness in upper limbs (HP:0008997); excluded: Proximal muscle weakness in lower limbs (HP:0008994); excluded: Disturbed sensory perception (HP:0010524)


In [10]:
encoder.output_individuals_as_phenopackets(individual_list=individuals)

We output 6 GA4GH phenopackets to the directory phenopackets


# HPOA file

In [11]:
ingestor = PhenopacketIngestor(indir="phenopackets")
ppkt_d = ingestor.get_phenopacket_dictionary()
ppkt_list = list(ppkt_d.values())
builder = HpoaTableBuilder(phenopacket_list=ppkt_list)
PMID = "PMID:36454683"
hpoa_table_creator = builder.autosomal_recessive(PMID).build()

[pyphetools] Ingested 6 GA4GH phenopackets.
We found a total of 15 unique HPO terms
Extracted disease: Neuronopathy, distal hereditary motor, autosomal recessive 9 (OMIM:620402)


In [12]:
df = hpoa_table_creator.get_dataframe()
df.head()

,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:620402,"Neuronopathy, distal hereditary motor, autosomal recessive 9",HP:0008959,Distal upper limb muscle weakness,,,3/3,,,,,PMID:37077559,PCS,ORCID:0000-0002-0736-9199
1,OMIM:620402,"Neuronopathy, distal hereditary motor, autosomal recessive 9",HP:0009053,Distal lower limb muscle weakness,,,3/3,,,,,PMID:37077559,PCS,ORCID:0000-0002-0736-9199
2,OMIM:620402,"Neuronopathy, distal hereditary motor, autosomal recessive 9",HP:0007210,Lower limb amyotrophy,,,3/3,,,,,PMID:37077559,PCS,ORCID:0000-0002-0736-9199
3,OMIM:620402,"Neuronopathy, distal hereditary motor, autosomal recessive 9",HP:0008997,Proximal muscle weakness in upper limbs,,,0/3,,,,,PMID:37077559,PCS,ORCID:0000-0002-0736-9199
4,OMIM:620402,"Neuronopathy, distal hereditary motor, autosomal recessive 9",HP:0008994,Proximal muscle weakness in lower limbs,,,0/3,,,,,PMID:37077559,PCS,ORCID:0000-0002-0736-9199


In [13]:
hpoa_table_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620402.tab
